# Marketing Agent: Rule-Based AI for Promotional Content

This project builds a rule-based AI agent that helps generate creative marketing content using a mix of prompt engineering and generative models.

### Project Tasks

The agent supports the following three tasks:

1. Product Description
   - Given a product name, return a short marketing-friendly product description.
   - Uses a pre-defined dictionary of products.

2. Marketing Image Generation (Text-to-Image)
   - Given a product name, generate a promotional-style image using a DALL·E-like image generator.
   - Uses prompt templates to describe the product visually.

3. Style Transfer for Marketing Image (Image-to-Image)
   - Given a marketing image of product X and the name of product Y, generate a marketing image for Y that **preserves the style** of X.

### Agent Architecture

We implement a rule-based agent that parses user input and routes it to the appropriate tool (function) based on task intent. This contrasts with LLM-based agents by using deterministic logic, making it ideal for simple logic to choose tools


### Learning Outcomes

- Understand the structure of a hybrid AI system (dictionary lookup + generative tools)
- Practice designing and testing prompts for image Generation Task
- Learn to build and extend rule-based agents using simple routing logic
- Evaluation using A/B testing


## Setting Up

In this section, we will:

- Install required libraries (`openai`, `langchain`)
- Import necessary Python modules
- Securely access the OpenAI API key using Colab Secrets


In [1]:
# Install core dependencies
!pip install openai langchain langchain_community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


In [2]:
# Imports
from openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI as LangOpenAI
from IPython.display import Image, display
import os
from google.colab import userdata


In [3]:
# Securely fetch OpenAI API key from Colab Secrets
openai_api_key = userdata.get("OPENAI_API_KEY")
assert openai_api_key is not None, "API key not found in secrets!"

# Initialize OpenAI client for image generation
client = OpenAI(api_key=openai_api_key)


## Tools

In this section, we define all the tools (functions) that the agent will use. Each tool performs a specific task and can be called directly or via the agent.
- Tool 1: Given product name get product description
- Tool 2: Generate image from product name using prompt
- Tool 3: Generate image of product Y in the style of image X

### Tool 1: Product Description

This tool takes a product name as input and returns a pre-defined marketing description for that product.

We simulate a backend product metadata system using a simple Python dictionary.  
If the product is not found, we return a helpful fallback message.

In [5]:
# Dictionary of predefined product descriptions (Apple-style reworded)
product_dict = {
    "SmartPhone": "A premium smartphone featuring edge-to-edge OLED display, advanced dual-camera system, and ultra-fast performance powered by the latest neural chip.",
    "WirelessPods": "Compact wireless earbuds with active noise cancellation, adaptive EQ, and seamless connectivity across all your devices.",
    "Laptop": "A lightweight professional-grade laptop with a high-resolution retina display, all-day battery life, and next-gen silicon performance.",
    "ImmersiveHeadset": "A spatial computing headset with ultra-clear displays, hand gesture input, and real-time 3D rendering for next-level productivity and entertainment."
}

In [6]:
# Tool 1: Fetch product description
def get_product_description(product_name: str) -> str:
    return product_dict.get(
        product_name,
        f"'{product_name}' not found in the product database."
    )

### Tool 2: Generate Marketing Image from Product Name

This tool takes a product name, retrieves its description using Tool 1, and then constructs a detailed image generation prompt to create a marketing-style image.

We use prompt engineering techniques to shape how the product is visualized. The final prompt is sent to the OpenAI image generation API (DALL·E 2).

This is the first point in the project where prompt design directly affects the quality of the output.

> Example Input: `"SmartPhone X"`
> Output: Generated image of the phone on a clean tech-oriented backdrop.


In [7]:
# Base prompt template for marketing image
# Optimize this prompt
image_prompt_template = PromptTemplate(
    input_variables=["product_name", "product_description"],
    template=(
        "Create a high-quality promotional product image of a {product_name}. "
        "The image should showcase: {product_description}. "

    )
)

In [8]:
def generate_marketing_image(product_name: str, size="512x512") -> str:

    #Get Product Description
    product_description = get_product_description(product_name)

    # Handle missing product gracefully
    if "not found in the product database" in product_description:
        print(f" No description available for '{product_name}'. Skipping image generation.")
        return None  # or a placeholder image URL

    # Generate Prompt
    image_prompt = image_prompt_template.format(
        product_name=product_name,
        product_description=product_description
    )

    #Connect the DALLE model with the prompt and get image url
    response = client.images.generate(
        model="dall-e-2",
        prompt=image_prompt,
        n=1,
        size=size
    )

    return response.data[0].url


### Tool 3: Style Transfer - Generate Product Y Image in the Style of Product X

This tool takes:

- A marketing image URL of product X
- The name of product Y

It generates a new marketing image for product Y that preserves the visual style of product X.


In [9]:
# Prompt template for style transfer
style_transfer_prompt_template = PromptTemplate(
    input_variables=["product_y", "product_y_desc", "product_x_image_url"],
    template=(
        "Create a promotional product image for {product_y} that mimics the style, lighting, and composition "
        "of this reference image: {product_x_image_url}. "
        "Product description: {product_y_desc}. "
        "Maintain the same mood and minimalistic tech style."
    )
)

In [10]:
def generate_style_transfer_image(product_x_image_url: str, product_y: str, size="512x512") -> str:
    # Get description of product Y
    product_y_desc = get_product_description(product_y)

    # Handle missing product gracefully
    if "not found in the product database" in product_y_desc:
        print(f"No description available for '{product_y}'. Skipping image generation.")
        return None

    # Format prompt
    prompt = style_transfer_prompt_template.format(
        product_y=product_y,
        product_y_desc=product_y_desc,
        product_x_image_url=product_x_image_url
    )

    # Generate image
    response = client.images.generate(
        model="dall-e-2",
        prompt=prompt,
        n=1,
        size=size
    )
    return response.data[0].url


## Rule-Based Agent

In this section, we define a simple **rule-based agent** that interprets user input and decides which tool to use:

---

### Supported Tasks:

- Get product description  
  → Calls Tool 1 (`get_product_description`)
- Generate marketing image
  → Calls Tool 2 (`generate_marketing_image`)
- Style transfer from product X to product Y
  → Calls Tool 3 (`generate_style_transfer_image`)

---

This agent does not use an LLM — it uses hardcoded rules and keyword matching.  
This structure makes the logic transparent and predictable, which is useful for debugging and teaching control flow.


In [11]:
def marketing_agent(user_input: str, reference_image_url: str = None) -> str:
    input_lower = user_input.lower()

    # Rule 1: Description request
    if "description" in input_lower or "describe" in input_lower:
        for product in product_dict:
            if product.lower() in input_lower:
                return get_product_description(product)

        return "No known product mentioned in the request."

    # Rule 2: Generate marketing image (only if no reference image is provided)
    elif ("generate" in input_lower and 'image' in input_lower  ) and reference_image_url is None:
        for product in product_dict:
            if product.lower() in input_lower:
                image_url = generate_marketing_image(product)
                if image_url:
                    display(Image(url=image_url))
                    return f" Marketing image generated for '{product}'"
                else:
                    return "Could not generate image."

        return "No known product mentioned for image generation."

    # Rule 3: Style transfer (requires reference image)
    elif "style of" in input_lower and "for" in input_lower and reference_image_url:
        product_x = None
        product_y = None
        for product in product_dict:
            if f"style of {product.lower()}" in input_lower:
                product_x = product
            if f"for {product.lower()}" in input_lower:
                product_y = product

        if product_y:
            new_image_url = generate_style_transfer_image(reference_image_url, product_y)
            if new_image_url:
                display(Image(url=new_image_url))
                return f" Generated image of '{product_y}' in style of '{product_x}'"
            else:
                return "Could not generate transferred image."
        else:
            return "Product Y not found in input."

    return " I'm not sure what you're asking. Try using keywords like 'description', 'generate image', or 'style of X for Y'."



## Evaluating the Agent

In this section, we test our rule-based agent by sending different types of input queries and observing how it routes the requests to different tools.

---

###  What We're Testing

- Tool 1: Fetching a product description
- Tool 2: Generating a marketing image
- Tool 3: Generating a new image of product Y in the style of product X

This gives us a chance to test:
- Rule-based input parsing
- Prompt generation
- Visual results of different prompt types

We'll also reuse the result of one tool (product X image) as input to another (style transfer).


In [12]:
# Tool 1: Description only
print(marketing_agent("Can you give me a description of SmartPhone X?"))

A premium smartphone featuring edge-to-edge OLED display, advanced dual-camera system, and ultra-fast performance powered by the latest neural chip.


In [ ]:
# Tool 2: Image generation
print(marketing_agent("Generate image of WirelessPods"))

In [ ]:
# Tool 2: Another product
print(marketing_agent("generate image for Laptop M"))

In [20]:
# Tool 3: Style transfer (simulate reference image)

# Generate an image of SmartPhone Ultra first
reference_url = generate_marketing_image( "SmartPhone")


In [ ]:
print(" Reference image for style transfer:")
display(Image(url=reference_url))
# Now use that for generating image of another product in same style
print(marketing_agent("Generate image in style of SmartPhone Ultra for ImmersiveHeadset X", reference_image_url=reference_url))

## A/B Testing Prompts: Group Activity

### What is A/B Testing?

A/B testing is a method of comparing two versions of something (in this case, prompts) to determine which performs better.

- **A Prompt** → Original prompt
- **B Prompt** → Modified version of the prompt (e.g., wording, style, tone)
- The output of each is evaluated based on visual quality, alignment to task, and creativity.

This process helps refine prompts to improve AI performance systematically, rather than by intuition alone.

### Group Activity Overview

We have 6 students. You’ll be divided into **2 groups of 3**:

- Group A: Will take the original image prompt (used in Tool 2) and optimize it for style, specificity, or branding
- Group B: Will do the same, but in a different direction (e.g., focus on setting, lighting, mood)

---

###  Step-by-Step Instructions

1. Each group edits the original prompt template** from Tool 2:
"Create a high-quality promotional product image of a {product_name}.
The image should showcase: {product_description}.
Use a clean, modern background suitable for tech advertising. Studio lighting, minimalistic setting."


2. Replace the `image_prompt_template` in the code with your group's version.

3. Run the updated prompt on the same product (e.g., `"ProLaptop M"`).

4. Save the image result (or copy the URL) as Version A or B.

5. Swap results** with the other group.

6. Each group evaluates the other group’s image using this rubric:

| Criteria             | Rating (1–5) |
|----------------------|-------------|
| Visual quality       |             |
| Marketing appeal     |             |
| Clarity of product   |             |
| Creativity / novelty |             |

---

### Discussion

- What prompt changes had the biggest effect?
- Which wording helped or hurt?
- How would you iterate again based on what you observed?

This exercise builds prompt intuition by making you test, compare, and explain your design choices — just like you would in a real-world AI product.

---